In [1]:
%pwd

'f:\\datascienceproject\\research'

In [2]:
import os

In [3]:
os.chdir("../")
%pwd


'f:\\datascienceproject'

In [5]:
import pandas as pd

data=pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [7]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [8]:
data.shape

(1599, 12)

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
     root_dir:Path
     STATUS_FILE:str
     unzip_data_dir:Path
     all_schema:dict
   


In [37]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

In [38]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema =self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            
            local_data_file=config.local_data_file,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config

In [39]:
import os
from src.datascience import logger

In [44]:


class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e


In [45]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    print(f"Validation directory: {data_validation_config.root_dir}")
    print(f"Status file path: {data_validation_config.STATUS_FILE}")
    
    data_validation = DataValidation(config=data_validation_config)
    result = data_validation.validate_all_columns()
    
    print(f"Validation result: {result}")
    print("Check the status file at:", data_validation_config.STATUS_FILE)
    
except Exception as e:
    print(f"Error: {e}")
    raise e


[2025-11-18 20:20:10,661: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-18 20:20:10,714: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-18 20:20:10,734: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-18 20:20:10,777: INFO: common: created directory at: artifacts]
Error: 'NoneType' object has no attribute 'root_dir'


AttributeError: 'NoneType' object has no attribute 'root_dir'

In [42]:
import os

status_file = "artifacts/data_validation/status.txt"

if os.path.exists(status_file):
    print("✓ Status file exists!")
    with open(status_file, 'r') as f:
        print(f"Content: {f.read()}")
else:
    print("✗ Status file not found!")
    print("Checking if directory exists...")
    if os.path.exists("artifacts/data_validation"):
        print("Directory exists")
    else:
        print("Directory doesn't exist - validation didn't create it")


✗ Status file not found!
Checking if directory exists...
Directory exists


In [43]:
import pandas as pd
import os

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    break
                else:
                    validation_status = True
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(self.config.STATUS_FILE), exist_ok=True)
            
            # Write final validation status
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")
            
            print(f"Status file written to: {self.config.STATUS_FILE}")

            return validation_status
        
        except Exception as e:
            raise e


In [46]:
import os
import pandas as pd
from pathlib import Path

# Step 1: Check if the data file exists
data_file = "artifacts/data_ingestion/winequality-red.csv"
print("=" * 60)
print("STEP 1: Checking if data file exists")
print("=" * 60)
if os.path.exists(data_file):
    print(f"✓ Data file exists: {data_file}")
    df = pd.read_csv(data_file)
    print(f"  Rows: {len(df)}, Columns: {len(df.columns)}")
    print(f"  Column names: {list(df.columns)}")
else:
    print(f"✗ Data file NOT found: {data_file}")
    print("  You need to run data ingestion first!")

# Step 2: Check directory structure
print("\n" + "=" * 60)
print("STEP 2: Checking directory structure")
print("=" * 60)
validation_dir = "artifacts/data_validation"
if os.path.exists(validation_dir):
    print(f"✓ Validation directory exists: {validation_dir}")
else:
    print(f"✗ Validation directory doesn't exist: {validation_dir}")
    print("  Creating it now...")
    os.makedirs(validation_dir, exist_ok=True)
    print(f"✓ Created: {validation_dir}")

# Step 3: Manually create and test STATUS.txt
print("\n" + "=" * 60)
print("STEP 3: Testing STATUS.txt creation")
print("=" * 60)
status_file = "artifacts/data_validation/status.txt"
try:
    with open(status_file, 'w') as f:
        f.write("Test: Validation status file created successfully!")
    print(f"✓ Successfully wrote to: {status_file}")
    
    # Read it back
    with open(status_file, 'r') as f:
        content = f.read()
    print(f"✓ Successfully read back: {content}")
except Exception as e:
    print(f"✗ Error: {e}")

# Step 4: List all files in artifacts directory
print("\n" + "=" * 60)
print("STEP 4: Listing all files in artifacts")
print("=" * 60)
if os.path.exists("artifacts"):
    for root, dirs, files in os.walk("artifacts"):
        level = root.replace("artifacts", "").count(os.sep)
        indent = " " * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = " " * 2 * (level + 1)
        for file in files:
            print(f"{subindent}{file}")
else:
    print("✗ artifacts folder doesn't exist!")

print("\n" + "=" * 60)
print("DEBUGGING COMPLETE")
print("=" * 60)


STEP 1: Checking if data file exists
✓ Data file exists: artifacts/data_ingestion/winequality-red.csv
  Rows: 1599, Columns: 12
  Column names: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']

STEP 2: Checking directory structure
✓ Validation directory exists: artifacts/data_validation

STEP 3: Testing STATUS.txt creation
✓ Successfully wrote to: artifacts/data_validation/status.txt
✓ Successfully read back: Test: Validation status file created successfully!

STEP 4: Listing all files in artifacts
artifacts/
  data_ingestion/
    winequality-red.csv
  data_validation/
    status.txt

DEBUGGING COMPLETE
